In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

class LoanRepaymentModel:
    def __init__(self, model):
        self.model = model
        self.transformer = None
        self.scaler = None

    def load(self, train_path, test_path):
        train_data = pd.read_excel(train_path)
        test_data = pd.read_excel(test_path)
        data = pd.concat((train_data, test_data), ignore_index=True)
        return data

    def preprocess(self, data):
        target_column = 'loan_status'
        x = data.drop(columns=['loan_status', 'customer_id', 'transaction_date'])
        y = data[target_column]

        # One-hot encode categorical variables
        self.transformer = ColumnTransformer(
            transformers=[
                ('tnf1', OneHotEncoder(sparse_output=False, drop='first'),
                 ['sub_grade', 'term', 'home_ownership', 'purpose', 'application_type', 'verification_status'])
            ],
            remainder='passthrough'
        )
        x_encoded = self.transformer.fit_transform(x)

        # Scale numerical features only for Logistic Regression
        if isinstance(self.model, LogisticRegression):
            self.scaler = StandardScaler()
            x_scaled = self.scaler.fit_transform(x_encoded)
            return x_scaled, y

        return x_encoded, y

    def train(self, x_train, y_train):
        self.model.fit(x_train, y_train)

    def test(self, x_test, y_test):
        y_pred = self.model.predict(x_test)
        print("Accuracy (test):", accuracy_score(y_test, y_pred))
        print("Classification Report:\n", classification_report(y_test, y_pred))
        return y_pred

    def predict(self, x_new):
        x_transformed = self.transformer.transform(x_new)
        if isinstance(self.model, LogisticRegression):
            x_transformed = self.scaler.transform(x_transformed)
        return self.model.predict(x_transformed)

# Load and preprocess data
train_path = '/content/train_data.xlsx'
test_path = '/content/test_data.xlsx'

# Logistic Regression Model
logistic_model = LoanRepaymentModel(LogisticRegression(max_iter=100, solver='liblinear', C=1))
data = logistic_model.load(train_path, test_path)
data = data.sample(frac=0.5, random_state=42)  # Use 50% of data for faster execution
x, y = logistic_model.preprocess(data)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Train and test Logistic Regression
logistic_model.train(x_train, y_train)
logistic_model.test(x_test, y_test)

# Random Forest Model
rf_model = LoanRepaymentModel(RandomForestClassifier(n_estimators=50, max_depth=10, random_state=42))
data = rf_model.load(train_path, test_path)
x, y = rf_model.preprocess(data)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Train and test Random Forest
rf_model.train(x_train, y_train)
rf_model.test(x_test, y_test)

# Example of predictions on new data
sample_data = data.iloc[:5, :-1]  # Replace with actual new data
print("Predictions:", rf_model.predict(sample_data))


Accuracy (test): 0.7645710543549443
Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.29      0.39      3203
           1       0.79      0.93      0.85      9013

    accuracy                           0.76     12216
   macro avg       0.70      0.61      0.62     12216
weighted avg       0.74      0.76      0.73     12216

Accuracy (test): 0.754379502292076
Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.19      0.30      6589
           1       0.76      0.96      0.85     17843

    accuracy                           0.75     24432
   macro avg       0.71      0.58      0.57     24432
weighted avg       0.73      0.75      0.70     24432

Predictions: [1 1 1 1 1]
